# Documentation: Java Game Project
Wintersemester 2025/26 – Hochschule Bielefeld – 
Author: Manuel Borghardt, Darnell Borghardt, Joel Jantschik, Leonid Nikkel

---

## 1. Projektidee & Konzept
- Spielfigur sammelt Coins und weicht Hindernissen aus. Wobei die Coins und Hindernisse von oben fallen.
- Ziel: Spieler muss im ersten Level 20 Coins und im zweiten Level 50 Coins Einsammeln. Darf dabei aber nicht einmal vom Hinderniss getroffen werden.
- Steuerung mit Pfeiltasten und 'A' und 'D', es gibt zwei Level wie schon im Ziel erwähnt.

---

## 2. Entwicklungsumgebung
- Java-Version: SE 18
- Framework/Library: libGDX
- Tools: Visual Studio Code mit folgenden Extensions: Java, Extension Pack for Java, Jupyter Notebook

---

## 3. GUI & Leveldesign
### 3.1 Spielfeld
- Beschreibung: Fenster mit Spielfigur, Punktestand, Hintergrund.
- Code-Snippet:
```java
// Beispiel: Spielfeld initialisieren
GameWindow window = new GameWindow(800, 600);
window.show();